# Library




In [ ]:
pip install openclean-core

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 267 kB 8.4 MB/s 
     |████████████████████████████████| 109 kB 53.3 MB/s 
     |████████████████████████████████| 260 kB 52.7 MB/s 
     |████████████████████████████████| 132 kB 58.7 MB/s 
     |████████████████████████████████| 525 kB 54.1 MB/s 
     |████████████████████████████████| 181 kB 55.3 MB/s 
     |████████████████████████████████| 212 kB 57.1 MB/s 
     |████████████████████████████████| 149 kB 54.2 MB/s 
     |████████████████████████████████| 63 kB 1.4 MB/s 
     |████████████████████████████████| 594 kB 52.5 MB/s 
     |████████████████████████████████| 856 kB 46.9 MB/s 
     |████████████████████████████████| 4.0 MB 50.7 MB/s 
  Created wheel for datasize: filename=datasize-1.0.0-py2.py3-none-any.whl size=155047 sha256=686bc24c82a29e45ab0b278f229a3407e12637258792d1833716183671fd9480
  Stored in directory: /root/.cache/pip/wheels/f7

In [17]:
pip install geopy #use to convert coordinates to zipcode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
git_folder = 'INET-Team-2-F2022'
if not os.path.isdir(git_folder):
  !git clone https://github.com/gcivil-nyu-org/INET-Team-2-F2022
else:
  %cd /tree/develop/Data
  !git pull
  %cd ..

Cloning into 'INET-Team-2-F2022'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 17 (delta 0), reused 4 (delta 0), pack-reused 0
Unpacking objects: 100% (17/17), done.


In [ ]:
from openclean.pipeline import stream
from openclean.profiling.column import DefaultColumnProfiler

In [ ]:
from openclean.cluster.knn import knn_clusters, knn_collision_clusters
from openclean.function.similarity.base import SimilarityConstraint
from openclean.function.similarity.text import LevenshteinDistance
from openclean.function.token.ngram import NGrams
from openclean.function.value.threshold import GreaterThan
from openclean.operator.transform.update import update
import pandas as pd

# 2018 Central Park Squirrel Data

In [ ]:
url ="https://raw.githubusercontent.com/gcivil-nyu-org/INET-Team-2-F2022/develop/Data/2018_Central_Park_Squirrel_Census_-_Squirrel_Data.csv"
dataset1 = pd.read_csv(url)
ds1 = stream(dataset1)

print('Schema\n------')
for col in ds1.columns:
    print("  '{}'".format(col))
    
print('\n{} rows.'.format(ds1.count()))
print("There are {} rows and {} columns in the dataset.".format(ds1.count(),len(ds1.columns)))

Schema
------
  'X'
  'Y'
  'Unique Squirrel ID'
  'Hectare'
  'Shift'
  'Date'
  'Hectare Squirrel Number'
  'Age'
  'Primary Fur Color'
  'Highlight Fur Color'
  'Combination of Primary and Highlight Color'
  'Color notes'
  'Location'
  'Above Ground Sighter Measurement'
  'Specific Location'
  'Running'
  'Chasing'
  'Climbing'
  'Eating'
  'Foraging'
  'Other Activities'
  'Kuks'
  'Quaas'
  'Moans'
  'Tail flags'
  'Tail twitches'
  'Approaches'
  'Indifferent'
  'Runs from'
  'Other Interactions'
  'Lat/Long'

3023 rows.
There are 3023 rows and 31 columns in the dataset.


In [ ]:
ds1.head()


,X,Y,Unique Squirrel ID,Hectare,Shift,Date,Hectare Squirrel Number,Age,Primary Fur Color,Highlight Fur Color,...,Kuks,Quaas,Moans,Tail flags,Tail twitches,Approaches,Indifferent,Runs from,Other Interactions,Lat/Long
0,-73.956134,40.794082,37F-PM-1014-03,37F,PM,10142018,3,None,None,None,...,False,False,False,False,False,False,False,False,None,POINT (-73.9561344937861 40.7940823884086)
1,-73.968857,40.783783,21B-AM-1019-04,21B,AM,10192018,4,None,None,None,...,False,False,False,False,False,False,False,False,None,POINT (-73.9688574691102 40.7837825208444)
2,-73.974281,40.775534,11B-PM-1014-08,11B,PM,10142018,8,None,Gray,None,...,False,False,False,False,False,False,False,False,None,POINT (-73.97428114848522 40.775533619083)
3,-73.959641,40.790313,32E-PM-1017-14,32E,PM,10172018,14,Adult,Gray,None,...,False,False,False,False,False,False,False,True,None,POINT (-73.9596413903948 40.7903128889029)
4,-73.970268,40.776213,13E-AM-1017-05,13E,AM,10172018,5,Adult,Gray,Cinnamon,...,False,False,False,False,False,False,False,False,None,POINT (-73.9702676472613 40.7762126854894)
5,-73.968361,40.772591,11H-AM-1010-03,11H,AM,10102018,3,Adult,Cinnamon,White,...,False,False,False,False,True,False,True,False,None,POINT (-73.9683613516225 40.7725908847499)
6,-73.95412,40.793181,36H-AM-1010-02,36H,AM,10102018,2,Adult,Gray,None,...,False,False,False,False,False,False,False,False,None,POINT (-73.9541201789795 40.7931811701082)
7,-73.958269,40.791737,33F-AM-1008-02,33F,AM,10082018,2,Adult,Gray,None,...,False,False,False,False,False,False,True,False,None,POINT (-73.9582694312289 40.7917367820255)
8,-73.967429,40.782972,21C-PM-1006-01,21C,PM,10062018,1,Adult,Gray,None,...,False,False,False,True,True,False,False,False,None,POINT (-73.9674285955293 40.7829723919744)
9,-73.97225,40.774288,11D-AM-1010-03,11D,AM,10102018,3,Adult,Gray,Cinnamon,...,False,False,False,False,False,False,True,False,None,POINT (-73.9722500196844 40.7742879599026)


In [ ]:
# https://github.com/gcivil-nyu-org/INET-Team-2-F2022/blob/develop/Data/DOF_Condominium_Comparable_Rental_Income_in_NYC.csv

Schema
------
  'X'
  'Y'
  'Unique Squirrel ID'
  'Hectare'
  'Shift'
  'Date'
  'Hectare Squirrel Number'
  'Age'
  'Primary Fur Color'
  'Highlight Fur Color'
  'Combination of Primary and Highlight Color'
  'Color notes'
  'Location'
  'Above Ground Sighter Measurement'
  'Specific Location'
  'Running'
  'Chasing'
  'Climbing'
  'Eating'
  'Foraging'
  'Other Activities'
  'Kuks'
  'Quaas'
  'Moans'
  'Tail flags'
  'Tail twitches'
  'Approaches'
  'Indifferent'
  'Runs from'
  'Other Interactions'
  'Lat/Long'

2835241 rows.
There are 2835241 rows and 60 columns in the dataset.


In [ ]:
# ds2.head()

,X,Y,Unique Squirrel ID,Hectare,Shift,Date,Hectare Squirrel Number,Age,Primary Fur Color,Highlight Fur Color,...,Kuks,Quaas,Moans,Tail flags,Tail twitches,Approaches,Indifferent,Runs from,Other Interactions,Lat/Long
0,-73.9561344937861,40.7940823884086,37F-PM-1014-03,37F,PM,10142018,3,,,,...,false,false,false,false,false,false,false,false,,POINT (-73.9561344937861 40.7940823884086)
1,-73.9688574691102,40.7837825208444,21B-AM-1019-04,21B,AM,10192018,4,,,,...,false,false,false,false,false,false,false,false,,POINT (-73.9688574691102 40.7837825208444)
2,-73.9742811484852,40.775533619083,11B-PM-1014-08,11B,PM,10142018,8,,Gray,,...,false,false,false,false,false,false,false,false,,POINT (-73.97428114848522 40.775533619083)
3,-73.9596413903948,40.7903128889029,32E-PM-1017-14,32E,PM,10172018,14,Adult,Gray,,...,false,false,false,false,false,false,false,true,,POINT (-73.9596413903948 40.7903128889029)
4,-73.9702676472613,40.7762126854894,13E-AM-1017-05,13E,AM,10172018,5,Adult,Gray,Cinnamon,...,false,false,false,false,false,false,false,false,,POINT (-73.9702676472613 40.7762126854894)
5,-73.9683613516225,40.7725908847499,11H-AM-1010-03,11H,AM,10102018,3,Adult,Cinnamon,White,...,false,false,false,false,true,false,true,false,,POINT (-73.9683613516225 40.7725908847499)
6,-73.9541201789795,40.7931811701082,36H-AM-1010-02,36H,AM,10102018,2,Adult,Gray,,...,false,false,false,false,false,false,false,false,,POINT (-73.9541201789795 40.7931811701082)
7,-73.9582694312289,40.7917367820255,33F-AM-1008-02,33F,AM,10082018,2,Adult,Gray,,...,false,false,false,false,false,false,true,false,,POINT (-73.9582694312289 40.7917367820255)
8,-73.9674285955293,40.7829723919744,21C-PM-1006-01,21C,PM,10062018,1,Adult,Gray,,...,false,false,false,true,true,false,false,false,,POINT (-73.9674285955293 40.7829723919744)
9,-73.9722500196844,40.7742879599026,11D-AM-1010-03,11D,AM,10102018,3,Adult,Gray,Cinnamon,...,false,false,false,false,false,false,true,false,,POINT (-73.9722500196844 40.7742879599026)


#DOF Condominium Comparable Rental Income

In [ ]:
ur2 ="https://raw.githubusercontent.com/gcivil-nyu-org/INET-Team-2-F2022/develop/Data/DOF_Condominium_Comparable_Rental_Income_in_NYC.csv"
dataset2 = pd.read_csv(ur2)
ds2 = stream(dataset2)

print('Schema\n------')
for col in ds2.columns:
    print("  '{}'".format(col))
    
print('\n{} rows.'.format(ds2.count()))
print("There are {} rows and {} columns in the dataset.".format(ds2.count(),len(ds2.columns)))

Schema
------
  'Boro-Block-Lot'
  'Condo Section'
  'Address'
  'Neighborhood'
  'Building Classification'
  'Total Units'
  'Year Built'
  'Gross SqFt'
  'Estimated Gross Income'
  'Gross Income per SqFt'
  'Estimated Expense'
  'Expense per SqFt'
  'Net Operating Income'
  'Full Market Value'
  'Market Value per SqFt'
  'Boro-Block-Lot 1'
  'Address 1'
  'Neighborhood 1'
  'Building Classification 1'
  'Total Units 1'
  'Year Built 1'
  'Gross SqFt 1'
  'Estimated Gross Income 1'
  'Gross Income per SqFt 1'
  'Estimated Expense 1'
  'Expense per SqFt 1'
  'Net Operating Income 1'
  'Full Market Value 1'
  'Market Value per SqFt 1'
  'Distance from Condo in miles'
  'Boro-Block-Lot 2'
  'Address 2'
  'Neighborhood 2'
  'Building Classification 2'
  'Total Units 2'
  'Year Built 2'
  'Gross SqFt 2'
  'Estimated Gross Income 2'
  'Gross Income per SqFt 2'
  'Estimated Expense 2'
  'Expense per SqFt 2'
  'Net Operating Income 2'
  'Full Market Value 2'
  'Market Value per SqFt 2'
  'Dis

In [ ]:
ds2.head()

,Boro-Block-Lot,Condo Section,Address,Neighborhood,Building Classification,Total Units,Year Built,Gross SqFt,Estimated Gross Income,Gross Income per SqFt,...,Gross SqFt 3,Estimated Gross Income 3,Gross Income per SqFt 3,Estimated Expense 3,Expense per SqFt 3,Net Operating Income 3,Full Market Value 3,Market Value per SqFt 3,Distance from Condo in miles 2,Report Year
0,1-00576-7501,0003-R1,60 WEST 13 STREET,GREENWICH VILLAGE-CENTRAL,R4 -ELEVATOR,70,1966.0,82017,4452703,54.29,...,27108.0,1471693.0,54.29,571708.0,21.09,899985.0,7318000.0,269.96,0.35,2019
1,1-01271-7501,0007-R2,1360 6 AVENUE,MIDTOWN WEST,R4 -ELEVATOR,183,1963.0,141738,7113830,50.19,...,39510.0,1987748.0,50.31,527854.0,13.36,1459894.0,9322000.0,235.94,0.63,2019
2,1-00894-7501,0009-R1,77 PARK AVENUE,MURRAY HILL,R4 -ELEVATOR,109,1924.0,158571,7329152,46.22,...,20784.0,960636.0,46.22,374112.0,18.0,586524.0,3423000.0,164.69,0.22,2019
3,1-00631-7501,0018-R1,712 GREENWICH STREET,GREENWICH VILLAGE-WEST,R9 -CONDOPS,20,1910.0,53943,2132906,39.54,...,37064.0,1461804.0,39.44,504070.0,13.6,957734.0,14265000.0,384.87,0.19,2019
4,1-00868-7501,0019-R1,35 EAST 38 STREET,MURRAY HILL,R4 -ELEVATOR,113,1961.0,88230,4288860,48.61,...,41603.0,2022322.0,48.61,497572.0,11.96,1524750.0,7628000.0,183.35,0.22,2019
5,1-00571-7501,0027-R1,7 EAST 13 STREET,GREENWICH VILLAGE-CENTRAL,R9 -CONDOPS,39,1928.0,42384,1666115,39.31,...,9135.0,379651.0,41.56,122318.0,13.39,257333.0,2485000.0,272.03,0.25,2019
6,1-01505-7501,0028-R1,55 EAST 93 STREET,UPPER EAST SIDE (79-96),R4 -ELEVATOR,29,1980.0,23232,1049622,45.18,...,20000.0,910600.0,45.53,276000.0,13.8,634600.0,5277000.0,263.85,0.46,2019
7,1-01245-7501,0029-R1,309 WEST 83 STREET,UPPER WEST SIDE (79-96),R9 -CONDOPS,17,1920.0,25216,1130938,44.85,...,11610.0,508170.0,43.77,155806.0,13.42,352364.0,2438000.0,209.99,0.28,2019
8,1-01310-7501,0030-R1,141 EAST 55 STREET,MIDTOWN CBD,R4 -ELEVATOR,87,1956.0,70773,3281036,46.36,...,58325.0,2719695.0,46.63,458435.0,7.86,2261260.0,19379000.0,332.26,0.52,2019
9,1-01141-7501,0031-R1,104 WEST 70 STREET,UPPER WEST SIDE (59-79),R4 -ELEVATOR,84,1903.0,95930,5217633,54.39,...,14025.0,738416.0,52.65,189057.0,13.48,549359.0,3433000.0,244.78,0.36,2019


# NYPD Arrest Data 

In [ ]:
ur3 ="https://raw.githubusercontent.com/gcivil-nyu-org/INET-Team-2-F2022/develop/Data/NYPD_Arrest_Data__Year_to_Date_.csv"
dataset3 = pd.read_csv(ur3)
ds3 = stream(dataset3)

print('Schema\n------')
for col in ds3.columns:
    print("  '{}'".format(col))
    
print('\n{} rows.'.format(ds3.count()))
print("There are {} rows and {} columns in the dataset.".format(ds3.count(),len(ds3.columns)))

Schema
------
  'ARREST_KEY'
  'ARREST_DATE'
  'PD_CD'
  'PD_DESC'
  'KY_CD'
  'OFNS_DESC'
  'LAW_CODE'
  'LAW_CAT_CD'
  'ARREST_BORO'
  'ARREST_PRECINCT'
  'JURISDICTION_CODE'
  'AGE_GROUP'
  'PERP_SEX'
  'PERP_RACE'
  'X_COORD_CD'
  'Y_COORD_CD'
  'Latitude'
  'Longitude'
  'New Georeferenced Column'

93238 rows.
There are 93238 rows and 19 columns in the dataset.


In [18]:
ds3.head()

,ARREST_KEY,ARREST_DATE,PD_CD,PD_DESC,KY_CD,OFNS_DESC,LAW_CODE,LAW_CAT_CD,ARREST_BORO,ARREST_PRECINCT,JURISDICTION_CODE,AGE_GROUP,PERP_SEX,PERP_RACE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,New Georeferenced Column
0,239406147,01/20/2022,None,(null),None,(null),CPL5700600,9,Q,113,3,25-44,M,BLACK,1046367,186986,40.679701,-73.776047,POINT (-73.77604735 40.67970059)
1,239828064,01/28/2022,None,(null),None,(null),CPL5700600,9,Q,113,3,45-64,M,BLACK,1046367,186986,40.679701,-73.776047,POINT (-73.77604735 40.67970059)
2,241020842,02/21/2022,None,(null),None,(null),CPL5700600,9,Q,102,0,25-44,M,BLACK,1032501,198800,40.712206,-73.825952,POINT (-73.825952 40.712206)
3,240475883,02/11/2022,157.0,RAPE 1,104.0,RAPE,PL 1303501,F,B,50,0,25-44,M,BLACK,1010914,260940,40.882862,-73.903574,POINT (-73.9035744897024 40.8828621313214)
4,241818674,03/09/2022,None,(null),None,(null),CPL5700600,9,Q,114,0,18-24,F,BLACK,1007694,219656,40.769552,-73.915361,POINT (-73.915361 40.769552)
5,243189844,04/07/2022,None,(null),None,(null),CPL5700600,9,Q,105,0,25-44,M,BLACK,1057766,203992,40.726284,-73.73476,POINT (-73.73476 40.726284)
6,242181223,03/16/2022,None,(null),None,(null),CPL5700600,9,Q,102,0,25-44,F,BLACK,1031727,199223,40.713373,-73.82874,POINT (-73.82874 40.713373)
7,242520065,03/23/2022,None,(null),None,(null),CPL5700600,9,Q,102,72,25-44,M,WHITE HISPANIC,1032501,198800,40.712206,-73.825952,POINT (-73.825952 40.712206)
8,244307669,04/27/2022,579.0,(null),None,(null),PL 2224002,F,S,120,0,25-44,M,WHITE,966190,160182,40.606321,-74.065039,POINT (-74.065039 40.606321)
9,242479164,03/22/2022,263.0,"ARSON 2,3,4",114.0,ARSON,PL 1501001,F,S,122,71,25-44,M,WHITE,958052,148351,40.573828,-74.094302,POINT (-74.094302 40.573828)


In [23]:
ds3_sample = dataset3[0:10]
ds3_sample

,ARREST_KEY,ARREST_DATE,PD_CD,PD_DESC,KY_CD,OFNS_DESC,LAW_CODE,LAW_CAT_CD,ARREST_BORO,ARREST_PRECINCT,JURISDICTION_CODE,AGE_GROUP,PERP_SEX,PERP_RACE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,New Georeferenced Column
0,239406147,01/20/2022,NaN,(null),NaN,(null),CPL5700600,9,Q,113,3,25-44,M,BLACK,1046367,186986,40.679701,-73.776047,POINT (-73.77604735 40.67970059)
1,239828064,01/28/2022,NaN,(null),NaN,(null),CPL5700600,9,Q,113,3,45-64,M,BLACK,1046367,186986,40.679701,-73.776047,POINT (-73.77604735 40.67970059)
2,241020842,02/21/2022,NaN,(null),NaN,(null),CPL5700600,9,Q,102,0,25-44,M,BLACK,1032501,198800,40.712206,-73.825952,POINT (-73.825952 40.712206)
3,240475883,02/11/2022,157.0,RAPE 1,104.0,RAPE,PL 1303501,F,B,50,0,25-44,M,BLACK,1010914,260940,40.882862,-73.903574,POINT (-73.9035744897024 40.8828621313214)
4,241818674,03/09/2022,NaN,(null),NaN,(null),CPL5700600,9,Q,114,0,18-24,F,BLACK,1007694,219656,40.769552,-73.915361,POINT (-73.915361 40.769552)
5,243189844,04/07/2022,NaN,(null),NaN,(null),CPL5700600,9,Q,105,0,25-44,M,BLACK,1057766,203992,40.726284,-73.734760,POINT (-73.73476 40.726284)
6,242181223,03/16/2022,NaN,(null),NaN,(null),CPL5700600,9,Q,102,0,25-44,F,BLACK,1031727,199223,40.713373,-73.828740,POINT (-73.82874 40.713373)
7,242520065,03/23/2022,NaN,(null),NaN,(null),CPL5700600,9,Q,102,72,25-44,M,WHITE HISPANIC,1032501,198800,40.712206,-73.825952,POINT (-73.825952 40.712206)
8,244307669,04/27/2022,579.0,(null),NaN,(null),PL 2224002,F,S,120,0,25-44,M,WHITE,966190,160182,40.606321,-74.065039,POINT (-74.065039 40.606321)
9,242479164,03/22/2022,263.0,"ARSON 2,3,4",114.0,ARSON,PL 1501001,F,S,122,71,25-44,M,WHITE,958052,148351,40.573828,-74.094302,POINT (-74.094302 40.573828)


In [24]:
import geopy
geo_locator = geopy.Nominatim(user_agent='1234')

def coord_to_zipcode(point): # (lati, longti)
    r = geo_locator.reverse(point)
    return r.raw['address']['postcode']

In [26]:
result = ds3_sample.apply(lambda row: coord_to_zipcode((row[16],row[17])), axis=1)
result

0    11434
1    11434
2    11415
3    10463
4    11103
5    11428
6    11415
7    11415
8    10305
9    10306
dtype: object

# Parks Properties

In [ ]:
ur4 ="https://raw.githubusercontent.com/gcivil-nyu-org/INET-Team-2-F2022/develop/Data/Parks_Properties.csv"
dataset4 = pd.read_csv(ur4)
ds4 = stream(dataset4)

print('Schema\n------')
for col in ds4.columns:
    print("  '{}'".format(col))
    
print('\n{} rows.'.format(ds4.count()))
print("There are {} rows and {} columns in the dataset.".format(ds4.count(),len(ds4.columns)))

Schema
------
  'ACQUISITIONDATE'
  'ACRES'
  'ADDRESS'
  'BOROUGH'
  'CLASS'
  'COMMUNITYBOARD'
  'COUNCILDISTRICT'
  'DEPARTMENT'
  'EAPPLY'
  'GISOBJID'
  'GISPROPNUM'
  'GlobalID'
  'JURISDICTION'
  'LOCATION'
  'MAPPED'
  'NAME311'
  'NYS_ASSEMBLY'
  'NYS_SENATE'
  'OBJECTID'
  'OMPPROPID'
  'PARENTID'
  'PERMIT'
  'PERMITDISTRICT'
  'PERMITPARENT'
  'PIP_RATABLE'
  'PRECINCT'
  'RETIRED'
  'SIGNNAME'
  'SUBCATEGORY'
  'TYPECATEGORY'
  'URL'
  'US_CONGRESS'
  'WATERFRONT'
  'ZIPCODE'
  'multipolygon'

2030 rows.
There are 2030 rows and 35 columns in the dataset.


In [ ]:
ds4.head()

,ACQUISITIONDATE,ACRES,ADDRESS,BOROUGH,CLASS,COMMUNITYBOARD,COUNCILDISTRICT,DEPARTMENT,EAPPLY,GISOBJID,...,PRECINCT,RETIRED,SIGNNAME,SUBCATEGORY,TYPECATEGORY,URL,US_CONGRESS,WATERFRONT,ZIPCODE,multipolygon
0,1937-06-01 00:00:00.0000000,249.389,None,Q,PARK,401403404406,1921222429,Q-03,Grand Central Parkway Ext,100000441.0,...,114.0,False,Grand Central Parkway Extension,EXWY,Parkway,http://www.nycgovparks.org/parks/Q084A/,614.0,True,11103113671136811369113701137111375,MULTIPOLYGON (((-73.8587476480729 40.767414466...
1,1948-09-23 00:00:00.0000000,0.037,None,X,PARK,205,14,X-05,Townsend Walk,100005179.0,...,46.0,False,Townsend Walk,STRIP,Strip,http://www.nycgovparks.org/parks/X148C4/,15.0,False,10453,MULTIPOLYGON (((-73.91265543859022 40.84584198...
2,2005-04-29 00:00:00.0000000,0.23,None,B,PARK,316,41,B-16,Marcus Garvey Tenant's Assoc. Garden,100004498.0,...,73.0,False,Marcus Garvey Tenant's Assoc. Garden,Greenthumb,Garden,http://www.nycgovparks.org/parks/B510/,9.0,False,11212,MULTIPOLYGON (((-73.91598050092 40.66927372409...
3,2002-11-20 00:00:00.0000000,0.074,None,B,PARK,303,36,B-03,Shiloh Garden,100004396.0,...,79.0,False,Shiloh Garden,Greenthumb,Garden,http://www.nycgovparks.org/parks/B502/,8.0,False,11216,MULTIPOLYGON (((-73.94519019107632 40.68617686...
4,None,0.185,None,Q,PARK,405,30,Q-05,Whitefish Triangle,100000196.0,...,104.0,False,Whitefish Triangle,Neighborhood Plgd,Triangle/Plaza,http://www.nycgovparks.org/parks/Q360G/,6.0,False,11378,MULTIPOLYGON (((-73.89868804397167 40.72512780...
5,1909-03-09 00:00:00.0000000,2.49,None,X,PARK,202,17,X-02,Joseph Rodman Drake Park & Enslaved African Bu...,100003884.0,...,41.0,False,Joseph Rodman Drake Park & Enslaved African Bu...,Neighborhood Park,Cemetery,http://www.nycgovparks.org/parks/X015/,15.0,False,10474,MULTIPOLYGON (((-73.88323029594491 40.80982262...
6,1967-10-18 00:00:00.0000000,111.163,None,Q,PARK,413,31,Q-13,Hook Creek Park,100000424.0,...,105.0,False,Hook Creek Park,Large Park,Nature Area,http://www.nycgovparks.org/parks/Q466/,5.0,True,1155911422,MULTIPOLYGON (((-73.73905463825886 40.65284962...
7,2018-02-09 00:00:00.0000000,0.054,None,M,PARK,103,2,M-03,None,100036978.0,...,9.0,False,Sage Garden,None,Garden,None,12.0,False,10009,MULTIPOLYGON (((-73.98082609012458 40.72295590...
8,1870-08-16 00:00:00.0000000,29.888,None,M,PARK,109,7,M-09,Morningside Park,100005044.0,...,26.0,False,Morningside Park,Large Park,Community Park,http://www.nycgovparks.org/parks/M056/,13.0,False,10026,MULTIPOLYGON (((-73.95845947997974 40.80298748...
9,1935-01-09 00:00:00.0000000,0.717,1300 SPOFFORD AVENUE,X,PARK,202,17,X-02,Hunts PointPlayground,100005079.0,...,41.0,False,Hunts Point Playground,JOP,Jointly Operated Playground,http://www.nycgovparks.org/parks/X113/,15.0,False,10474,MULTIPOLYGON (((-73.88619730395624 40.81384928...


# NYC Community Based Organizations

In [ ]:
ur5 ="https://raw.githubusercontent.com/gcivil-nyu-org/INET-Team-2-F2022/develop/Data/Parks_Properties.csv"
dataset5 = pd.read_csv(ur5)
ds5 = stream(dataset5)

print('Schema\n------')
for col in ds5.columns:
    print("  '{}'".format(col))
    
print('\n{} rows.'.format(ds5.count()))
print("There are {} rows and {} columns in the dataset.".format(ds5.count(),len(ds5.columns)))

Schema
------
  'ACQUISITIONDATE'
  'ACRES'
  'ADDRESS'
  'BOROUGH'
  'CLASS'
  'COMMUNITYBOARD'
  'COUNCILDISTRICT'
  'DEPARTMENT'
  'EAPPLY'
  'GISOBJID'
  'GISPROPNUM'
  'GlobalID'
  'JURISDICTION'
  'LOCATION'
  'MAPPED'
  'NAME311'
  'NYS_ASSEMBLY'
  'NYS_SENATE'
  'OBJECTID'
  'OMPPROPID'
  'PARENTID'
  'PERMIT'
  'PERMITDISTRICT'
  'PERMITPARENT'
  'PIP_RATABLE'
  'PRECINCT'
  'RETIRED'
  'SIGNNAME'
  'SUBCATEGORY'
  'TYPECATEGORY'
  'URL'
  'US_CONGRESS'
  'WATERFRONT'
  'ZIPCODE'
  'multipolygon'

2030 rows.
There are 2030 rows and 35 columns in the dataset.


In [ ]:
ds5.head()

,ACQUISITIONDATE,ACRES,ADDRESS,BOROUGH,CLASS,COMMUNITYBOARD,COUNCILDISTRICT,DEPARTMENT,EAPPLY,GISOBJID,...,PRECINCT,RETIRED,SIGNNAME,SUBCATEGORY,TYPECATEGORY,URL,US_CONGRESS,WATERFRONT,ZIPCODE,multipolygon
0,1937-06-01 00:00:00.0000000,249.389,None,Q,PARK,401403404406,1921222429,Q-03,Grand Central Parkway Ext,100000441.0,...,114.0,False,Grand Central Parkway Extension,EXWY,Parkway,http://www.nycgovparks.org/parks/Q084A/,614.0,True,11103113671136811369113701137111375,MULTIPOLYGON (((-73.8587476480729 40.767414466...
1,1948-09-23 00:00:00.0000000,0.037,None,X,PARK,205,14,X-05,Townsend Walk,100005179.0,...,46.0,False,Townsend Walk,STRIP,Strip,http://www.nycgovparks.org/parks/X148C4/,15.0,False,10453,MULTIPOLYGON (((-73.91265543859022 40.84584198...
2,2005-04-29 00:00:00.0000000,0.23,None,B,PARK,316,41,B-16,Marcus Garvey Tenant's Assoc. Garden,100004498.0,...,73.0,False,Marcus Garvey Tenant's Assoc. Garden,Greenthumb,Garden,http://www.nycgovparks.org/parks/B510/,9.0,False,11212,MULTIPOLYGON (((-73.91598050092 40.66927372409...
3,2002-11-20 00:00:00.0000000,0.074,None,B,PARK,303,36,B-03,Shiloh Garden,100004396.0,...,79.0,False,Shiloh Garden,Greenthumb,Garden,http://www.nycgovparks.org/parks/B502/,8.0,False,11216,MULTIPOLYGON (((-73.94519019107632 40.68617686...
4,None,0.185,None,Q,PARK,405,30,Q-05,Whitefish Triangle,100000196.0,...,104.0,False,Whitefish Triangle,Neighborhood Plgd,Triangle/Plaza,http://www.nycgovparks.org/parks/Q360G/,6.0,False,11378,MULTIPOLYGON (((-73.89868804397167 40.72512780...
5,1909-03-09 00:00:00.0000000,2.49,None,X,PARK,202,17,X-02,Joseph Rodman Drake Park & Enslaved African Bu...,100003884.0,...,41.0,False,Joseph Rodman Drake Park & Enslaved African Bu...,Neighborhood Park,Cemetery,http://www.nycgovparks.org/parks/X015/,15.0,False,10474,MULTIPOLYGON (((-73.88323029594491 40.80982262...
6,1967-10-18 00:00:00.0000000,111.163,None,Q,PARK,413,31,Q-13,Hook Creek Park,100000424.0,...,105.0,False,Hook Creek Park,Large Park,Nature Area,http://www.nycgovparks.org/parks/Q466/,5.0,True,1155911422,MULTIPOLYGON (((-73.73905463825886 40.65284962...
7,2018-02-09 00:00:00.0000000,0.054,None,M,PARK,103,2,M-03,None,100036978.0,...,9.0,False,Sage Garden,None,Garden,None,12.0,False,10009,MULTIPOLYGON (((-73.98082609012458 40.72295590...
8,1870-08-16 00:00:00.0000000,29.888,None,M,PARK,109,7,M-09,Morningside Park,100005044.0,...,26.0,False,Morningside Park,Large Park,Community Park,http://www.nycgovparks.org/parks/M056/,13.0,False,10026,MULTIPOLYGON (((-73.95845947997974 40.80298748...
9,1935-01-09 00:00:00.0000000,0.717,1300 SPOFFORD AVENUE,X,PARK,202,17,X-02,Hunts PointPlayground,100005079.0,...,41.0,False,Hunts Point Playground,JOP,Jointly Operated Playground,http://www.nycgovparks.org/parks/X113/,15.0,False,10474,MULTIPOLYGON (((-73.88619730395624 40.81384928...
